# Sparkify Project - Music Streaming Data

This project aims to leverage approximate **27,000,000** Sparkify user activity records to predict user churn, identifying those most likely to discontinue their engagement. The process involves the retrieval of the complete **12GB dataset** from **AWS S3** and its subsequent processing on an **AWS EMR cluster**. We employ **Spark** for data wrangling, feature engineering, model training, and hyperparameter tuning.

### Import libraries from PySpark and Python

### Connect to AWS EMR cluster

In [1]:
# Start Spark Session

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
3,application_1693877095915_0005,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
# Spark SQL
from pyspark.sql import SparkSession
from pyspark.sql.functions import isnan, count, when, desc, udf, col, sort_array, asc, avg, lit, max, min, countDistinct
import pyspark.sql.functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import IntegerType

# Spark ML
from pyspark.ml import Pipeline
from pyspark.ml.feature import MinMaxScaler, OneHotEncoder, StringIndexer, VectorAssembler
from pyspark.ml.classification import LogisticRegression, GBTClassifier, NaiveBayes, LinearSVC, RandomForestClassifier
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator

# Python basic
import datetime
import numpy as np

import warnings
warnings.filterwarnings("ignore")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Create a Spark session & Load 12GB full data from AWS S3

In [3]:
# Download java first# Create a SparkSession with the application name "sparkify".
# SparkSession is the entry point for interacting with the Spark cluster.
# It requires Java Runtime Environment (JRE) to run, as Spark is built using Scala and Java.

## Create spark session
spark = SparkSession \
    .builder \
    .appName("Sparkify") \
    .getOrCreate()

## Read in full sparkify dataset
event_data = "s3n://udacity-dsnd/sparkify/sparkify_event_data.json"
df = spark.read.json(event_data)
df.head()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Row(artist='Popol Vuh', auth='Logged In', firstName='Shlok', gender='M', itemInSession=278, lastName='Johnson', length=524.32934, level='paid', location='Dallas-Fort Worth-Arlington, TX', method='PUT', page='NextSong', registration=1533734541000, sessionId=22683, song='Ich mache einen Spiegel - Dream Part 4', status=200, ts=1538352001000, userAgent='"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"', userId='1749042')

In [4]:
df.head()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Row(artist='Popol Vuh', auth='Logged In', firstName='Shlok', gender='M', itemInSession=278, lastName='Johnson', length=524.32934, level='paid', location='Dallas-Fort Worth-Arlington, TX', method='PUT', page='NextSong', registration=1533734541000, sessionId=22683, song='Ich mache einen Spiegel - Dream Part 4', status=200, ts=1538352001000, userAgent='"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"', userId='1749042')

In [5]:
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)

In [6]:
print('Total activity records:', df.count())
print('Total columns:', len(df.columns))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Total activity records: 26259199
Total columns: 18

In [7]:
df.select("auth").dropDuplicates().sort("auth").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+
|      auth|
+----------+
| Cancelled|
|     Guest|
| Logged In|
|Logged Out|
+----------+

### Clean data

In [8]:
# Exclude "not signed up" user
df.filter(df['userId']=='').count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0

In [9]:
df = df.filter(df['userId']!='')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
# Extract Cancelled User and Logged in user
df_clean = df.filter((col('auth') == 'Cancelled') | (col('auth') == 'Logged In'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
# check missing data
# missing_value_counts = df_clean.select([sum(col(column).isNull().cast("int")).alias(column + "_missing") for column in df_clean.columns])
# for row in missing_value_counts.collect():
#     for column, missing_count in row.asDict().items():
#         print(f"{column}: {missing_count}")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
df_clean.filter(df_clean.userId == '').count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0

In [20]:
df_clean.filter(df_clean.sessionId == '').count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0

0 missing data (`UserId` and `SessionId`)

Please be aware that the 'artist' and 'songs' columns have more missing values, accounting for about 20% of rows. This is due to the nature of the records, which involve actions such as logging into the service and accessing the homepage.

### Define Feature & Feature Engineering

In [21]:
# Create outcome column named "cancel"
df_clean = df_clean.withColumn('cancel', when(df_clean['auth'] == 'Cancelled', 1).otherwise(0).cast(IntegerType()))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
# Convert gender to 1/0. 1 = male
male = udf(lambda x: 1 if x == 'M' else 0, IntegerType())
df_clean = df_clean.withColumn('Male', male('gender'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Convert time series data

In [24]:
df_clean = df_clean.withColumn("date", F.to_date(F.from_unixtime(col('ts')/lit(1000))))
df_clean = df_clean.withColumn('date_created', F.to_date(F.from_unixtime(col('registration')/lit(1000))))
df_clean = df_clean.withColumn('user_age', F.datediff(col('date'), col('date_created')))
df_clean.describe('user_age').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-----------------+
|summary|         user_age|
+-------+-----------------+
|  count|         25480720|
|   mean| 65.7917132639894|
| stddev|41.19433609714086|
|    min|              -23|
|    max|              411|
+-------+-----------------+

### Feature Generation from 'page' column

In [25]:
df_clean.select("page").dropDuplicates().sort("page").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+
|                page|
+--------------------+
|               About|
|          Add Friend|
|     Add to Playlist|
|              Cancel|
|Cancellation Conf...|
|           Downgrade|
|               Error|
|                Help|
|                Home|
|              Logout|
|            NextSong|
|         Roll Advert|
|       Save Settings|
|            Settings|
|    Submit Downgrade|
|      Submit Upgrade|
|         Thumbs Down|
|           Thumbs Up|
|             Upgrade|
+--------------------+

In [26]:
# Create `Thumbs Down` feature which means negative feedback from user
Thumbs_Down = udf(lambda x: 1 if x == 'Thumbs Down' else 0, IntegerType())
df_clean = df_clean.withColumn('Thumbs_Down', Thumbs_Down('page'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
# Create `songs` feature to calculate number of songs from users
NextSong = udf(lambda x: 1 if x == 'NextSong' else 0, IntegerType())
df_clean = df_clean.withColumn('songs', NextSong('page'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
# Create `downgrades` to show if the user ever downgraded
downgrade = udf(lambda x: 1 if x == 'Submit Downgrade' else 0, IntegerType())
df_clean = df_clean.withColumn('downgraded', downgrade('page'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
# Create `paid` to show if the user ever downgraded
paid = udf(lambda x: 1 if x == 'paid' else 0, IntegerType())
df_clean = df_clean.withColumn('paid', paid('level'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Create new dataframe for churn prediction - Spark SQL

In [30]:
# user information
df_clean.createOrReplaceTempView("df_clean_temp")
users = spark.sql("""
    SELECT
        userId AS userId,
        male AS gender,
        MAX(user_age) AS user_age,
        MAX(paid) AS paid,
        MAX(downgraded) AS downgraded,
        MAX(cancel) AS canceled
    FROM
        df_clean_temp
    GROUP BY
        userId, male, date_created
""")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
# Interaction information by user and session
df_clean.createOrReplaceTempView("df_clean_temp")
interaction = spark.sql("""
    SELECT
        userId AS userId,
        sessionId,
        MIN(date) AS start_date,
        MAX(date) AS end_date,
        SUM(Thumbs_Down) AS Thumbs_Down,
        SUM(songs) AS songs_num,
        COUNT(DISTINCT artist) AS artists_num
    FROM
        df_clean_temp
    GROUP BY
        userId, sessionId
""")

interaction.take(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(userId='1410808', sessionId=26032, start_date=datetime.date(2018, 10, 1), end_date=datetime.date(2018, 10, 2), Thumbs_Down=1, songs_num=32, artists_num=32)]

In [32]:
# Interaction infomration by user
interaction.createOrReplaceTempView("measures_temp")
interaction_user = spark.sql("""
    SELECT
        userId AS userId,
        COUNT(sessionId) AS total_session,
        AVG(Thumbs_Down) AS avg_Thumbs_Down,
        AVG(songs_num) AS avg_songs,
        AVG(artists_num) AS avg_artists_num,
        AVG(session_gap) AS avg_session_gap
    FROM (
        SELECT
            *,
            LAG(end_date) OVER (PARTITION BY userId ORDER BY start_date) AS prev_end_date,
            DATEDIFF(start_date, LAG(end_date) OVER (PARTITION BY userId ORDER BY start_date)) AS session_gap
        FROM
            measures_temp
    ) AS temp
    GROUP BY
        userId
""")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
print('users: ',users.count(),' interaction_user: ',interaction_user.count())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

users:  22277  interaction_user:  22277

In [34]:
df_model = users.join(interaction_user, on = 'userId')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Modeling

In [35]:
# Assemble Features
assembler = VectorAssembler(inputCols=['gender','user_age','paid',\
                                       'downgraded','total_session',\
                                      'avg_Thumbs_Down','avg_artists_num','avg_session_gap','avg_songs'],\
                            outputCol='raw_features',handleInvalid = 'skip')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
df_model.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- userId: string (nullable = true)
 |-- gender: integer (nullable = true)
 |-- user_age: integer (nullable = true)
 |-- paid: integer (nullable = true)
 |-- downgraded: integer (nullable = true)
 |-- canceled: integer (nullable = true)
 |-- total_session: long (nullable = false)
 |-- avg_Thumbs_Down: double (nullable = true)
 |-- avg_songs: double (nullable = true)
 |-- avg_artists_num: double (nullable = true)
 |-- avg_session_gap: double (nullable = true)

### Logistic regression

In [45]:
# ML pipeline
scaler = MinMaxScaler(inputCol="raw_features", outputCol="features")
preprocess_stages = [assembler, scaler]

rf = RandomForestClassifier(labelCol="canceled", featuresCol="features")
lr = LogisticRegression(labelCol="canceled", featuresCol="features")

pipeline = Pipeline(stages=[*preprocess_stages, lr])

train_data, test_data = df_model.randomSplit([0.7, 0.3], seed=42)

param_grid = ParamGridBuilder() \
    .addGrid(lr.regParam, [0.01, 0.1]) \
    .build()

evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction", labelCol="canceled", metricName="areaUnderROC")

cross_val = CrossValidator(estimator=pipeline,
                           estimatorParamMaps=param_grid,
                           evaluator=evaluator,
                           numFolds=5,
                           seed=42)

model = cross_val.fit(train_data)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [44]:
# Evaluate
test_results = model.transform(test_data)
area_under_roc = evaluator.evaluate(test_results)


best_model = model.bestModel
best_lr_model = best_model.stages[-1] 
best_reg_param = best_lr_model.getOrDefault("regParam")

print(f"Best regParam: {best_reg_param}")
print(f"Area Under ROC: {area_under_roc}")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Best regParam: 0.01
Area Under ROC: 0.8130784048962069

### Random Forest

In [335]:
# ML pipeline
pipeline = Pipeline(stages=[*preprocess_stages, rf])

train_data, test_data = df_model.randomSplit([0.7, 0.3], seed=42)

param_grid = ParamGridBuilder() \
    .addGrid(rf.maxDepth, [5, 10]) \
    .build()

evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction", labelCol="canceled", metricName="areaUnderROC")

cross_val = CrossValidator(estimator=pipeline,
                           estimatorParamMaps=param_grid,
                           evaluator=evaluator,
                           numFolds=5,
                           seed=42)

rf_model = cross_val.fit(train_data)

# evaluate
test_results = rf_model.transform(test_data)
area_under_roc = evaluator.evaluate(test_results)

In [336]:
best_model = rf_model.bestModel
best_rf_model = best_model.stages[-1]
best_rf_param = best_rf_model.getOrDefault("maxDepth")

print(f"Best maxDepth: {best_rf_param}")

print(f"Area Under ROC: {area_under_roc}")

Best maxDepth: 10
Area Under ROC: 0.787267080745342
